In [1]:
%matplotlib notebook

In [2]:
import os, sys, glob

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import multiprocessing as mp

import elephant
from neo.core import AnalogSignal
import quantities as pq
import xarray as xr

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

In [3]:
exp_data_dir = '/Users/atleeskelandrimehaug/Dropbox (Simula)/scripts_for_bmtk/data_files/exp'

In [4]:
z_data = np.linspace(0, 1000E-6, 51) * pq.m       # [m] length of electrode
diameter = 1600E-6 * pq.m                              # [m] source diameter
h = 20E-6 * pq.m                                  # [m] distance between channels
sigma = 0.3* pq.S / pq.m                         # [S/m] or [1/(ohm*m)] extracellular conductivity
sigma_top = 0.3* pq.S / pq.m                     # [S/m] or [1/(ohm*m)] conductivity on top of cortex
order_hann = int(2*z_data[-1]/h)                  # order for hann-filter smoothing

delta_input = {
        'sigma' : sigma,
        'sigma_top' : sigma,
        'f_type' : 'gaussian',  # gaussian filter
        'f_order' : (3, 1),     # 3-point filter, sigma = 1.
    }

In [5]:
def compute_csd(session, diam=500*pq.um, method='DeltaiCSD', struct='cortex'):
    
    channel_vertical_pos = session[struct]['channel_vertical_pos']
    coord = [[i] for i in channel_vertical_pos]*pq.um
    sf = session[struct]['sf']
    
    
    lfp_mean = session[struct]['lfp_mean']
    neo_lfp = AnalogSignal(lfp_mean.T, units='V', sampling_rate=sf*pq.Hz)
    csd = elephant.current_source_density.estimate_csd(neo_lfp, coords=coord, method=method, diam=diam, sigma=0.3*pq.S/pq.m)
    csd_mean = xr.DataArray(csd.T, dims=lfp_mean.dims, coords=lfp_mean.coords, name='csd_mean',)
    
    lfp = session[struct]['lfp']
    single_csd = np.zeros_like(lfp.data)
    for t in range(len(lfp.presentation_id)):
        neo_lfp = AnalogSignal(lfp.data[:,t,:].T, units='V', sampling_rate=sf*pq.Hz)
        csd = elephant.current_source_density.estimate_csd(neo_lfp, coords=coord, method=method, diam=diam)
        single_csd[:,t,:] = csd.T
    csd = xr.DataArray(single_csd, dims=lfp.dims, coords=lfp.coords, name='csd',)
    
    s = dict()
    s['csd_mean'] = csd_mean
    s['csd'] = csd
    s['channel_vertical_pos'] = channel_vertical_pos
    s['time'] = session[struct]['time']
    s['sf'] = sf
    return s

In [8]:
#path = os.path.join('..', '..', 'processed_data', 'lfp', 'flash_white', '*')
#file_list = glob.glob(path)

file_list = glob.glob(os.path.join(exp_data_dir,'*_lfp.npy'))
print(file_list[0])
file_list.sort()

print(file_list[0])



for f in file_list[0:1]:
    try:
        X = np.load(f, allow_pickle=True).item()
        print(X['name'])
        output = dict()
        output['cortex'] = compute_csd(X, diam=diameter, method='DeltaiCSD', struct='cortex')
        #output['cortex'] = compute_csd(X, diam=1600*pq.um, method='DeltaiCSD', struct='cortex')
        #output['lgn'] = compute_csd(X, diam=500*pq.um, method='DeltaiCSD', struct='lgn')
        output['name'] = X['name']
    
        name = str(output['name'])+'_csd.npy'
        #path = os.path.join('..', '..', 'processed_data', 'csd', 'flash_white', name)
        path = os.path.join(exp_data_dir,name)
        #np.save(path, output)
    except:
        print('Failed')

/Users/atleeskelandrimehaug/Dropbox (Simula)/scripts_for_bmtk/data_files/exp/715093703_lfp.npy
/Users/atleeskelandrimehaug/Dropbox (Simula)/scripts_for_bmtk/data_files/exp/715093703_lfp.npy
715093703
discrete filter coefficients: 
b = [ 0.607 1.000 0.607 ],                
a = [ 2.213 ]
discrete filter coefficients: 
b = [ 0.607 1.000 0.607 ],                
a = [ 2.213 ]
discrete filter coefficients: 
b = [ 0.607 1.000 0.607 ],                
a = [ 2.213 ]
discrete filter coefficients: 
b = [ 0.607 1.000 0.607 ],                
a = [ 2.213 ]
discrete filter coefficients: 
b = [ 0.607 1.000 0.607 ],                
a = [ 2.213 ]
discrete filter coefficients: 
b = [ 0.607 1.000 0.607 ],                
a = [ 2.213 ]
discrete filter coefficients: 
b = [ 0.607 1.000 0.607 ],                
a = [ 2.213 ]
discrete filter coefficients: 
b = [ 0.607 1.000 0.607 ],                
a = [ 2.213 ]
Failed


In [7]:
path = os.path.join('..', '..', 'processed_data', 'lfp', 'flash_black', '*')

file_list = glob.glob(path)
print(file_list[0])
file_list.sort()

print(file_list[0])

for f in file_list:
    try:
        X = np.load(f, allow_pickle=True).item()
        print(X['name'])
        output = dict()
        output['cortex'] = compute_csd(X, diam=diameter, method='DeltaiCSD', struct='cortex')
        #output['cortex'] = compute_csd(X, diam=1600*pq.um, method='DeltaiCSD', struct='cortex')
    #     output['lgn'] = compute_csd(X, diam=500*pq.um, method='DeltaiCSD', struct='lgn')
        output['name'] = X['name']
    
        name = str(output['name'])+'_csd.npy'
        path = os.path.join('..', '..', 'processed_data', 'csd', 'flash_black', name)
        np.save(path, output)
    except:
        print('Failed')

IndexError: list index out of range